### Пердобработка данных

In [970]:
#библиотеки
import pandas as pd
import numpy as np
import json
from requests import get

In [ ]:
#выгрузка по инн поставщиков
list_inn_supplierinn = [7735116621, 7729394758, 7731625191, 7702735506, 7722523002, 5024096727, 7709654648, 7710499161, 
                        9723041340, 7717629606, 7811625045, 7810797732, 7840501541, 3664115445, 7704767623, 7715631511, 
                        7722585721, 7714842686, 5904159516, 7805109081, 7734649680, 7424022032, 7703608910, 7842464310, 
                        7724833987, 9705138050, 7704030780, 7729498813, 5260146265]

json_contracts_zaka = []

for inn in list_inn_supplierinn:
    req = get(f"http://openapi.clearspending.ru/restapi/v3/contracts/search//?supplierinn={inn}")
    if str(req) != '<Response [404]>':
        data = req.json()
        json_contracts_zaka.append(pd.DataFrame.from_dict(data["contracts"]["data"]))

#выгрузка по инн заказчиков
list_inn_customerinn = [7710878000, 7701903677, 7706074737, 7704055136, 7702005066, 7701014621]

json_contracts_post = []

for inn in list_inn_customerinn:
    req = get(f"http://openapi.clearspending.ru/restapi/v3/contracts/search//?customerinn={inn}")
    if str(req) != '<Response [404]>':
        data = req.json()
        json_contracts_post.append(pd.DataFrame.from_dict(data["contracts"]["data"]))

#собираем в черновой датафрейм
pd_customerinn = pd.DataFrame(pd.concat(json_contracts_post))
pd_supplierinn = pd.DataFrame(pd.concat(json_contracts_zaka))

In [968]:
#cобираем читый датафрейм из черновго датафрейма поставщиков
supplier_data = pd.DataFrame(columns=['name_commodity', 'price', 'quantity', 'number_contract', 'inn_supplier', 
                                      'name_supplier', 'inn_customer', 'name_customer', 'date_auction', 
                                      'date_signing_contract', 'address'])

for i in range(len(pd_supplierinn)):
    if len(pd.DataFrame(pd_supplierinn.iloc[i]['products'])) > 1:
        for j in range(len(pd.DataFrame(pd_supplierinn.iloc[i]['products']))):
            
            #название товара
            if 'name' in pd.DataFrame(pd_supplierinn.iloc[i]['products']).columns.tolist():
                name_commodity = pd.DataFrame(pd_supplierinn.iloc[i]['products'])[['name']].loc[j]['name']
            else:
                try:
                    if 'name' in pd.DataFrame(pd_supplierinn.iloc[i]['purchaseInfo'], index=[0]).columns.tolist():
                        name_commodity = pd_supplierinn.iloc[i]['purchaseInfo']['name']
                    else:
                        name_commodity = 'NA'
                except (RuntimeError, TypeError, NameError, ValueError):
                    name_commodity = 'NA'
                    
            #цена товара
            if 'price' in pd.DataFrame(pd_supplierinn.iloc[i]['products']).columns.tolist():
                price = pd.DataFrame(pd_supplierinn.iloc[i]['products'])[['price']].loc[j]['price']
            elif 'quantity' in pd.DataFrame(pd_supplierinn.iloc[i]['products']).columns.tolist():
                price = pd_supplierinn.iloc[i]['price'] / pd.DataFrame(pd_supplierinn.iloc[i]['products'])[['quantity']].astype('int').sum()[0]
            elif np.isnan(pd.DataFrame(pd_supplierinn.iloc[i]['products']).iloc[1][0]):
                price = 'NA'
            else:
                price = pd_supplierinn.iloc[i]['price']
            
            #количество товаров
            if 'quantity' in pd.DataFrame(pd_supplierinn.iloc[i]['products']).columns.tolist():
                quantity = pd.DataFrame(pd_supplierinn.iloc[i]['products'])[['quantity']].loc[j]['quantity']
            else: quantity = 'NA'
            
            #Номер контракта
            number_contract = pd_supplierinn.iloc[i]['regNum']
            
            #инн поставщика
            if len(pd_supplierinn.iloc[i]['suppliers']) == 1:
                inn_supplier = pd.DataFrame(pd_supplierinn.iloc[i]['suppliers'], index=[0])['inn'][0]
            else:
                inn_supplier = pd.DataFrame(pd_supplierinn.iloc[i]['suppliers'])['inn'][0]
            
            #название поставщика
            if len(pd_supplierinn.iloc[i]['suppliers']) == 1:
                name_supplier = pd.DataFrame(pd_supplierinn.iloc[i]['suppliers'], index=[0])['organizationName'][0]
            else:
                name_supplier = pd.DataFrame(pd_supplierinn.iloc[i]['suppliers'])['organizationName'][0]
                
            #инн заказчика
            inn_customer = pd.DataFrame(pd_supplierinn.iloc[i]['customer'], index=[0])['inn'][0]
            
            #сокращенное название заказчика
            name_customer = pd.DataFrame(pd_supplierinn.iloc[i]['customer'], index=[0])['fullName'][0]
            
            #дата аукциона
            date_auction = pd_supplierinn.iloc[i]['protocolDate']
            
            #дата подписания контракта
            date_signing_contract = pd_supplierinn.iloc[i]['publishDate']
            
            #адрес
            if 'legalAddress' in pd.DataFrame(pd_supplierinn.iloc[i]['customer'], index=[0]).columns.tolist():
                address = pd.DataFrame(pd_supplierinn.iloc[i]['customer'], index=[0])['legalAddress'][0]
                address = re.sub('^[0-9]+\,\sг\s', '', address)
                city = re.search('\w+',address).group()
            elif 'postalAddress' in pd.DataFrame(pd_supplierinn.iloc[i]['customer'], index=[0]).columns.tolist():
                address = pd.DataFrame(pd_supplierinn.iloc[i]['customer'], index=[0])['postalAddress'][0]
            else: address = 'NA'
            
            supplier_data = supplier_data.append({
                    'name_commodity': name_commodity,
                    'price': price,
                    'quantity': quantity,
                    'number_contract': number_contract, 
                    'inn_supplier': inn_supplier, 
                    'name_supplier': name_supplier,
                    'inn_customer': inn_customer,
                    'name_customer': name_customer, 
                    'date_auction': date_auction, 
                    'date_signing_contract': date_signing_contract, 
                    'address': address}, 
                   ignore_index=True)
    else:
        
        #название товара
        try:
            if 'name' in pd.DataFrame(pd_supplierinn.iloc[i]['purchaseInfo'], index=[0]).columns.tolist():
                name_commodity = pd_supplierinn.iloc[i]['purchaseInfo']['name']
            else:
                name_commodity = pd.DataFrame(pd_supplierinn.iloc[i]['products'])['name'][0]
        except (RuntimeError, TypeError, NameError, ValueError):
            name_commodity = pd.DataFrame(pd_supplierinn.iloc[i]['products'])['name'][0]
        
        #цена товара
        price = pd_supplierinn.iloc[i]['price']
        
        quantity = '1'
        
        #Номер контракта
        number_contract = pd_supplierinn.iloc[i]['regNum']
        
        #инн поставщика
        if len(pd_supplierinn.iloc[i]['suppliers']) == 1:
            inn_supplier = pd.DataFrame(pd_supplierinn.iloc[i]['suppliers'], index=[0])['inn'][0]
        else:
            inn_supplier = pd.DataFrame(pd_supplierinn.iloc[i]['suppliers'])['inn'][0]
        
        #название поставщика
        if len(pd_supplierinn.iloc[i]['suppliers']) == 1:
            name_supplier = pd.DataFrame(pd_supplierinn.iloc[i]['suppliers'], index=[0])['organizationName'][0]
        else:
            name_supplier = pd.DataFrame(pd_supplierinn.iloc[i]['suppliers'])['organizationName'][0]
        
        #инн заказчика
        inn_customer = pd.DataFrame(pd_supplierinn.iloc[i]['customer'], index=[0])['inn'][0]
        
        #сокращенное название заказчика
        name_customer = pd.DataFrame(pd_supplierinn.iloc[i]['customer'], index=[0])['fullName'][0]
        
        #дата аукциона
        date_auction = pd_supplierinn.iloc[i]['protocolDate']
        
        #дата подписания контракта
        date_signing_contract = pd_supplierinn.iloc[i]['publishDate']
        
        #адрес
        if 'legalAddress' in pd.DataFrame(pd_supplierinn.iloc[i]['customer'], index=[0]).columns.tolist():
            address = pd.DataFrame(pd_supplierinn.iloc[i]['customer'], index=[0])['legalAddress'][0]
            address = re.sub('^[0-9]+\,\sг\s', '', address)
            city = re.search('\w+',address).group()
        elif 'postalAddress' in pd.DataFrame(pd_supplierinn.iloc[i]['customer'], index=[0]).columns.tolist():
            address = pd.DataFrame(pd_supplierinn.iloc[i]['customer'], index=[0])['postalAddress'][0]
        else: address = 'NA'

        supplier_data = supplier_data.append({
                    'name_commodity': name_commodity,
                    'price': price,
                    'quantity': quantity,
                    'number_contract': number_contract, 
                    'inn_supplier': inn_supplier, 
                    'name_supplier': name_supplier,
                    'inn_customer': inn_customer,
                    'name_customer': name_customer, 
                    'date_auction': date_auction, 
                    'date_signing_contract': date_signing_contract, 
                    'address': address}, 
                   ignore_index=True)

In [971]:
#уточняю типы данных
#формат времени
supplier_data['date_signing_contract'] = pd.to_datetime(supplier_data['date_signing_contract'], format='%Y-%m-%dT%H:%M:%S', utc=True)
#формат цены
supplier_data["price"].replace({"NA": 0}, inplace=True)
supplier_data["price"] = supplier_data['price'].astype('float')
#формат количества
supplier_data["quantity"].replace({"NA": 0}, inplace=True)
supplier_data["quantity"] = supplier_data['quantity'].astype('float')

In [972]:
supplier_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1393 entries, 0 to 1392
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   name_commodity         1393 non-null   object             
 1   price                  1393 non-null   float64            
 2   quantity               1391 non-null   float64            
 3   number_contract        1384 non-null   object             
 4   inn_supplier           1393 non-null   object             
 5   name_supplier          1393 non-null   object             
 6   inn_customer           1393 non-null   object             
 7   name_customer          1393 non-null   object             
 8   date_auction           1225 non-null   object             
 9   date_signing_contract  1393 non-null   datetime64[ns, UTC]
 10  address                1393 non-null   object             
dtypes: datetime64[ns, UTC](1), float64(2), object(8)
memory 

In [ ]:
#собираем данные для покупателей в чистовую таблицу
customer_data = pd.DataFrame(columns=['name_commodity', 'price', 'quantity', 'number_contract', 'inn_supplier', 
                                      'name_supplier', 'inn_customer', 'name_customer', 'date_auction', 
                                      'date_signing_contract', 'address'])

for i in range(len(pd_customerinn)):
    if len(pd.DataFrame(pd_customerinn.iloc[i]['products'])) > 1:
        for j in range(len(pd.DataFrame(pd_customerinn.iloc[i]['products']))):
            
            #название товара
            if 'name' in pd.DataFrame(pd_customerinn.iloc[i]['products']).columns.tolist():
                name_commodity = pd.DataFrame(pd_customerinn.iloc[i]['products'])[['name']].loc[j]['name']
            else:
                try:
                    if 'name' in pd.DataFrame(pd_customerinn.iloc[i]['purchaseInfo'], index=[0]).columns.tolist():
                        name_commodity = pd_customerinn.iloc[i]['purchaseInfo']['name']
                    else:
                        name_commodity = 'NA'
                except (RuntimeError, TypeError, NameError, ValueError):
                    name_commodity = 'NA'
                    
            #цена товара
            if 'price' in pd.DataFrame(pd_customerinn.iloc[i]['products']).columns.tolist():
                price = pd.DataFrame(pd_customerinn.iloc[i]['products'])[['price']].loc[j]['price']
            elif 'quantity' in pd.DataFrame(pd_customerinn.iloc[i]['products']).columns.tolist():
                price = pd_customerinn.iloc[i]['price'] / pd.DataFrame(pd_customerinn.iloc[i]['products'])[['quantity']].astype('int').sum()[0]
            elif np.isnan(pd.DataFrame(pd_customerinn.iloc[i]['products']).iloc[1][0]):
                price = 'NA'
            else:
                price = pd_customerinn.iloc[i]['price']
            
            #количество товаров
            if 'quantity' in pd.DataFrame(pd_customerinn.iloc[i]['products']).columns.tolist():
                quantity = pd.DataFrame(pd_customerinn.iloc[i]['products'])[['quantity']].loc[j]['quantity']
            else: quantity = 'NA'
            
            #Номер контракта
            number_contract = pd_customerinn.iloc[i]['regNum']
            
            #инн поставщика
            try:
                if len(pd_customerinn.iloc[i]['suppliers']) == 1:
                    inn_supplier = pd.DataFrame(pd_customerinn.iloc[i]['suppliers'], index=[0])['inn'][0]
                else:
                    inn_supplier = pd.DataFrame(pd_customerinn.iloc[i]['suppliers'])['inn'][0]
            except (RuntimeError, TypeError, NameError, ValueError):
                inn_supplier = 'NA'

            #название поставщика
            try:
                if len(pd_customerinn.iloc[i]['suppliers']) == 1:
                    inn_supplier = pd.DataFrame(pd_customerinn.iloc[i]['suppliers'], index=[0])['organizationName'][0]
                else:
                    inn_supplier = pd.DataFrame(pd_customerinn.iloc[i]['suppliers'])['organizationName'][0]
            except (RuntimeError, TypeError, NameError, ValueError):
                inn_supplier = 'NA'
                
            #инн заказчика
            inn_customer = pd.DataFrame(pd_customerinn.iloc[i]['customer'], index=[0])['inn'][0]
            
            #сокращенное название заказчика
            name_customer = pd.DataFrame(pd_customerinn.iloc[i]['customer'], index=[0])['fullName'][0]
            
            #дата аукциона
            date_auction = pd_customerinn.iloc[i]['protocolDate']
            
            #дата подписания контракта
            date_signing_contract = pd_customerinn.iloc[i]['publishDate']
            
            #адрес
            if 'legalAddress' in pd.DataFrame(pd_customerinn.iloc[i]['customer'], index=[0]).columns.tolist():
                address = pd.DataFrame(pd_customerinn.iloc[i]['customer'], index=[0])['legalAddress'][0]
                address = re.sub('^[0-9]+\,\sг\s', '', address)
                city = re.search('\w+',address).group()
            elif 'postalAddress' in pd.DataFrame(pd_customerinn.iloc[i]['customer'], index=[0]).columns.tolist():
                address = pd.DataFrame(pd_customerinn.iloc[i]['customer'], index=[0])['postalAddress'][0]
            else: address = 'NA'
            
            customer_data = customer_data.append({
                    'name_commodity': name_commodity,
                    'price': price,
                    'quantity': quantity,
                    'number_contract': number_contract, 
                    'inn_supplier': inn_supplier, 
                    'name_supplier': name_supplier,
                    'inn_customer': inn_customer,
                    'name_customer': name_customer, 
                    'date_auction': date_auction, 
                    'date_signing_contract': date_signing_contract, 
                    'address': address}, 
                   ignore_index=True)
    else:
        
        #название товара
        try:
            if 'name' in pd.DataFrame(pd_customerinn.iloc[i]['purchaseInfo'], index=[0]).columns.tolist():
                name_commodity = pd_customerinn.iloc[i]['purchaseInfo']['name']
            else:
                name_commodity = pd.DataFrame(pd_customerinn.iloc[i]['products'])['name'][0]
        except (RuntimeError, TypeError, NameError, ValueError):
            name_commodity = pd.DataFrame(pd_customerinn.iloc[i]['products'])['name'][0]
        
        #цена товара
        price = pd_customerinn.iloc[i]['price']
        
        quantity = '1'
        
        #Номер контракта
        number_contract = pd_customerinn.iloc[i]['regNum']
        
        #инн поставщика
        try:
            if len(pd_customerinn.iloc[i]['suppliers']) == 1:
                inn_supplier = pd.DataFrame(pd_customerinn.iloc[i]['suppliers'], index=[0])['inn'][0]
            else:
                inn_supplier = pd.DataFrame(pd_customerinn.iloc[i]['suppliers'])['inn'][0]
        except (RuntimeError, TypeError, NameError, ValueError):
            inn_supplier = 'NA'
        
        #название поставщика
        try:
            if len(pd_customerinn.iloc[i]['suppliers']) == 1:
                inn_supplier = pd.DataFrame(pd_customerinn.iloc[i]['suppliers'], index=[0])['organizationName'][0]
            else:
                inn_supplier = pd.DataFrame(pd_customerinn.iloc[i]['suppliers'])['organizationName'][0]
        except (RuntimeError, TypeError, NameError, ValueError):
            inn_supplier = 'NA'
        
        #инн заказчика
        inn_customer = pd.DataFrame(pd_customerinn.iloc[i]['customer'], index=[0])['inn'][0]
        
        #сокращенное название заказчика
        name_customer = pd.DataFrame(pd_customerinn.iloc[i]['customer'], index=[0])['fullName'][0]
        
        #дата аукциона
        date_auction = pd_customerinn.iloc[i]['protocolDate']
        
        #дата подписания контракта
        date_signing_contract = pd_customerinn.iloc[i]['publishDate']
        
        #адрес
        if 'legalAddress' in pd.DataFrame(pd_customerinn.iloc[i]['customer'], index=[0]).columns.tolist():
            address = pd.DataFrame(pd_customerinn.iloc[i]['customer'], index=[0])['legalAddress'][0]
            address = re.sub('^[0-9]+\,\sг\s', '', address)
            city = re.search('\w+',address).group()
        elif 'postalAddress' in pd.DataFrame(pd_customerinn.iloc[i]['customer'], index=[0]).columns.tolist():
            address = pd.DataFrame(pd_customerinn.iloc[i]['customer'], index=[0])['postalAddress'][0]
        else: address = 'NA'

        customer_data = customer_data.append({
                    'name_commodity': name_commodity,
                    'price': price,
                    'quantity': quantity,
                    'number_contract': number_contract, 
                    'inn_supplier': inn_supplier, 
                    'name_supplier': name_supplier,
                    'inn_customer': inn_customer,
                    'name_customer': name_customer, 
                    'date_auction': date_auction, 
                    'date_signing_contract': date_signing_contract, 
                    'address': address}, 
                   ignore_index=True)

In [983]:
#уточняю типы данных
#формат времени
customer_data['date_signing_contract'] = pd.to_datetime(customer_data['date_signing_contract'], format='%Y-%m-%dT%H:%M:%S', utc=True)
#формат цены
customer_data["price"] = customer_data['price'].astype('float')
#формат количества
customer_data["quantity"].replace({"NA": 0}, inplace=True)
customer_data["quantity"] = customer_data['quantity'].astype('float')

In [984]:
customer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1037 entries, 0 to 1036
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   name_commodity         1037 non-null   object             
 1   price                  1037 non-null   float64            
 2   quantity               939 non-null    float64            
 3   number_contract        1037 non-null   object             
 4   inn_supplier           1037 non-null   object             
 5   name_supplier          1037 non-null   object             
 6   inn_customer           1037 non-null   object             
 7   name_customer          1037 non-null   object             
 8   date_auction           963 non-null    object             
 9   date_signing_contract  1037 non-null   datetime64[ns, UTC]
 10  address                1037 non-null   object             
dtypes: datetime64[ns, UTC](1), float64(2), object(8)
memory 

In [1008]:
full_data = pd.DataFrame(pd.concat([customer_data, supplier_data]))

### На выходе 

In [ ]:
#полные данные
full_data
#данные по инн заказчиков
customer_data
#данные по инн поставщиков
supplier_data

In [1011]:
#сохраняю в csv
full_data.to_csv(r'C:\Users\nikita\Documents\проект мониторинг\datasets\full_data.csv', index = False)
customer_data.to_csv(r'C:\Users\nikita\Documents\проект мониторинг\datasets\customer_data.csv', index = False)
supplier_data.to_csv(r'C:\Users\nikita\Documents\проект мониторинг\datasets\supplier_data.csv', index = False)